In [18]:
#coding:utf-8
%reload_ext autoreload 
%autoreload 2
import os
import sys

project_basedir = '..'
sys.path.append(project_basedir)
from cchess_zero import cbf
from cchess_zero import board_visualizer
from matplotlib import pyplot as plt
import os
from cchess_zero.gameboard import *
from net import resnet
import common
from gameplays import gameplay,players
from gameplays.analysis import get_km_json
import json
%matplotlib inline

In [58]:

fullpath = '../data/distributed/2018-07-24_23-26-51_161_mcts-mcts_net-net_b.cbf'
moves = cbf.cbf2move(fullpath)
fname = fullpath.split('/')[-1]
print(fname)
state = gameplay.GameState()

2018-07-24_23-26-51_161_mcts-mcts_net-net_b.cbf


In [59]:
from PIL import Image
import imageio
imgs_for_gif = []
state = gameplay.GameState()
statestr = 'RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c1/9/rnbakabnr'
for i in moves[:]: 
    statestr = GameBoard.sim_do_action(i,statestr)
    imgs_for_gif.append(board_visualizer.get_board_img(statestr))
    state.do_move(i)
imgs_for_gif = imgs_for_gif[-10:]
imageio.mimsave('../data/gif/{}-.gif'.format(fname[:-4]), imgs_for_gif,duration=0.5)

In [60]:
i,len(moves)

('d2d1', 107)

In [61]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [62]:
display_gif('../data/gif/{}-.gif'.format(fname[:-4]))

# ANALYSIS

In [7]:
network = resnet.get_model(os.path.join(project_basedir,
    'data/prepare_weight/2018-07-21_21-13-25'),
    common.board.create_uci_labels(),GPU_CORE=[None],FILTERS=128,NUM_RES_LAYERS=7)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from ../data/prepare_weight/2018-07-21_21-13-25


In [48]:
state.currentplayer

'b'

In [49]:
network_player = players.NetworkPlayer(state.currentplayer,network,n_playout=400)

In [50]:
network_player.make_move(state=state,actual_move=False)

('e9d9', -0.8296206136047841)

In [51]:
xx = network_player.mcts_policy._root._children
for i in xx:
    val = xx[i]
    print(i,val._n_visits)

e9d9 249
e9f9 145
e9e8 1
h3g3 1
h3i3 1
h3h2 1
f7e8 1


In [ ]:
val._n_visits

In [ ]:
datajson = get_km_json(network_player.mcts_policy,state.statestr,depth=40)

In [ ]:
with open('../data/mstc_vis/sample.km','w') as whdl:
    json.dump(datajson,whdl)